<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myProspects_Upside_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/quantvesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Mounted at /content/drive


In [2]:
# Import the function from the script
import myProspects_Upside_Analysis_v2

# Call the function to get the DataFrame
df_prospects_features = myProspects_Upside_Analysis_v2.get_prospects_features_v2()
cols = ['Symbol', 'Close', 'FTT',  'Dev%_200', 'Dev%_PE', 'Conviction', 'RSP', 'Today%', 'FTT%', 'ATH%','Gained%', 'Criteria', 'Strategy','LatestQtr', 'StarStock', 'Category', 'InFolio']
df_prospects_features = df_prospects_features[cols]

Run date time (IST): 2025-01-24 10:04:40
-------------------
qualified stocks: 173
with latest results: 164
still star stocks: 91


In [3]:
# x24/x25
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XY24', 'X25'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Category,InFolio
3,ADANIPORTS,1094.40,1583.0,-19.71,-12.40,M-LC,39.31,-0.87,44.65,45.30,2.60,XY24,NTT,1,1,MISC,NaN
29,BOROLTD,370.00,500.5,-6.33,NaN,NaN,44.51,-2.03,35.27,35.27,23.21,XY24,ATH,1,0,GLASS,0.0
7,AMBUJACEM,557.65,733.0,-7.83,47.57,M-LC,85.55,1.61,31.44,24.63,15.18,XY24,BTT,0,0,CEMENT,0.0
136,ROSSARI,745.70,965.0,-8.05,NaN,NaN,55.49,-1.00,29.41,110.36,11.25,XY24,NTT,1,1,CHEMICALS,0.0
161,UTIAMC,1214.00,1395.0,7.62,-7.77,M-SC,76.30,0.03,14.91,14.91,53.24,XY24,ATH,1,1,FINANCE,0.0
26,BIOCON,389.60,397.0,13.92,-33.46,M-MC,98.27,-1.94,1.90,22.32,58.75,XY24,NTT,1,1,PHARMA,0.0
92,JSWHL,18315.00,7487.0,86.74,170.03,H-SC,100.00,-2.00,-59.12,6.03,231.78,XY24,BTT,1,0,FINANCE,NaN


In [4]:
# watchlist
df_tmp = df_prospects_features[df_prospects_features['InFolio'] == -1].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Category,InFolio
8,ANGELONE,2365.00,3775.05,-10.21,-20.43,H-SC,4.62,-2.73,59.62,59.62,15.78,X40N,ATH,1,1,FINANCE,-1.0
0,3MINDIA,29429.90,40020.60,-12.35,-27.52,H-MC,54.91,-1.85,35.99,35.99,4.22,X40N,ATH,1,1,MISC,-1.0
150,TATACONSUM,990.60,1252.91,-8.45,17.52,H-MC,94.80,0.68,26.48,26.48,11.37,AR,ATH,1,0,FMCG,-1.0
128,PGHH,14429.90,18249.76,-10.11,-10.34,X-MC,73.99,-0.34,26.47,26.47,0.00,X40,ATH,0,0,FMCG,-1.0
30,BOSCHLTD,30685.00,38795.60,-8.21,9.21,X-LC,21.97,-0.99,26.43,26.43,36.34,X40N,ATH,1,1,AUTO,-1.0
76,ICICIGI,1814.65,2268.71,-3.83,-10.90,H-MC,69.94,-1.06,25.02,25.02,23.23,X40,ATH,1,1,INSURANCE,-1.0


In [5]:
# X-LC and H-LC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].isin(['X-LC','H-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Category,InFolio
19,BAJAJ-AUTO,8382.80,12666.40,-13.64,56.99,X-LC,59.25,-0.20,51.10,51.10,11.95,X40,ATH,1,1,AUTO,0.0
48,DLF,705.70,1031.70,-15.59,-14.28,H-LC,7.51,-1.31,46.20,46.20,0.00,AR,ATH,1,0,REALTY,NaN
122,OFSS,9913.50,12945.45,-4.66,95.34,H-LC,4.05,-1.42,30.58,30.58,58.75,X40N,ATH,1,1,IT,NaN
30,BOSCHLTD,30685.00,38795.60,-8.21,9.21,X-LC,21.97,-0.99,26.43,26.43,36.34,X40N,ATH,1,1,AUTO,-1.0
47,DIXON,15262.00,18940.15,17.06,9.26,H-LC,9.54,-1.30,24.10,24.10,160.41,X40N,ATH,1,1,IT,NaN
131,POLYCAB,6179.00,7541.35,-7.22,31.99,H-LC,12.14,-1.15,22.05,22.05,45.58,X40N,ATH,1,1,CABLES,NaN
116,NAUKRI,7463.50,9025.50,2.51,52.47,H-LC,10.40,-0.96,20.93,20.93,53.04,NaN,ATH,1,0,IT,NaN
97,KOTAKBANK,1877.30,2270.00,5.76,-39.90,X-LC,93.06,-0.93,20.92,17.48,21.51,X40,BTT,1,1,BANKS,0.0
88,ITC,441.00,522.75,-5.16,21.65,H-LC,37.28,0.18,18.54,18.54,11.27,X40,ATH,1,1,FMCG,NaN
39,CIPLA,1434.00,1680.50,-5.03,-12.35,H-LC,70.52,-1.18,17.19,17.19,9.80,NaN,ATH,1,1,PHARMA,NaN


In [6]:
# X-MC and X-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('X-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['X-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Category,InFolio
111,METROPOLIS,1847.60,3102.00,-9.21,7.21,X-SC,33.53,-1.23,67.89,85.74,18.54,XR,NTT,1,0,HEALTHCARE,0.0
14,ASTRAL,1471.10,2412.74,-24.79,-19.43,X-MC,10.98,-1.63,64.01,64.01,1.08,AR,ATH,1,0,PLASTICS,0.0
165,VINATIORGA,1707.35,2325.26,-7.81,-8.14,X-SC,83.24,-0.73,36.19,36.19,16.61,X40N,ATH,1,0,CHEMICALS,NaN
45,DEEPAKNTR,2318.10,3102.60,-12.29,23.75,X-MC,19.65,-2.16,33.84,33.84,12.53,AR,ATH,1,0,CHEMICALS,NaN
128,PGHH,14429.90,18249.76,-10.11,-10.34,X-MC,73.99,-0.34,26.47,26.47,0.00,X40,ATH,0,0,FMCG,-1.0
34,CAPLIPOINT,2139.15,2600.15,19.57,76.03,X-SC,20.81,-3.43,21.55,21.55,72.68,X40N,ATH,1,1,PHARMA,NaN
140,SHREECEM,26146.80,31310.07,1.16,11.91,X-MC,74.57,0.85,19.75,19.75,9.29,NaN,ATH,1,0,CEMENT,NaN
143,SRF,2569.40,2808.38,7.10,92.24,X-MC,97.69,-1.05,9.30,9.30,19.86,XR,ATH,1,0,CHEMICALS,0.0


In [7]:
# H-MC and H-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('H-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['H-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Category,InFolio
62,GLAND,1592.15,3442.00,-11.94,-6.19,H-SC,23.70,-1.87,116.19,168.33,0.00,XR,NTT,1,0,PHARMA,0.0
8,ANGELONE,2365.00,3775.05,-10.21,-20.43,H-SC,4.62,-2.73,59.62,59.62,15.78,X40N,ATH,1,1,FINANCE,-1.0
104,MGL,1285.00,1964.95,-15.35,-16.35,H-SC,86.71,-2.13,52.91,52.91,14.19,AR,ATH,1,1,GAS,NaN
70,HEROMOTOCO,4061.80,6190.55,-19.41,-6.35,H-MC,57.51,-0.94,52.41,52.41,0.92,AR,ATH,1,1,AUTO,NaN
40,CLEAN,1413.70,2137.00,-1.96,-8.49,H-SC,77.46,-1.31,51.16,86.25,10.81,XR,NTT,1,0,CHEMICALS,0.0
96,KAJARIACER,1035.70,1541.71,-19.28,-12.28,H-SC,34.10,-1.01,48.86,48.86,1.51,AR,ATH,1,1,CERAMICS,0.0
141,SONATSOFTW,580.30,852.37,-6.40,131.62,H-SC,47.40,-1.83,46.88,46.88,16.55,MGC,ATH,1,0,IT,NaN
63,GLAXO,2075.00,3047.48,-16.03,-7.74,H-MC,32.95,-0.60,46.87,46.87,12.79,X40,ATH,1,0,PHARMA,NaN
137,SANOFI,5345.40,7552.00,-23.46,27.14,H-SC,12.72,-0.27,41.28,88.44,0.00,X40,NTT,0,0,PHARMA,NaN
112,MOTILALOFS,708.80,1000.00,-3.83,7.02,H-MC,1.73,-2.41,41.08,116.90,102.38,X40N,NTT,1,1,FINANCE,NaN


In [8]:
# x40
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Category,InFolio
19,BAJAJ-AUTO,8382.80,12666.40,-13.64,56.99,X-LC,59.25,-0.20,51.10,51.10,11.95,X40,ATH,1,1,AUTO,0.0
63,GLAXO,2075.00,3047.48,-16.03,-7.74,H-MC,32.95,-0.60,46.87,46.87,12.79,X40,ATH,1,0,PHARMA,NaN
137,SANOFI,5345.40,7552.00,-23.46,27.14,H-SC,12.72,-0.27,41.28,88.44,0.00,X40,NTT,0,0,PHARMA,NaN
168,WHIRLPOOL,1615.95,2270.00,-14.36,-14.23,M-SC,9.54,0.51,40.47,64.09,32.62,X40,NTT,1,0,DURABLES,0.0
127,PFIZER,4719.95,6318.30,-7.74,-7.57,H-SC,75.14,-1.35,33.86,33.86,15.59,X40,ATH,1,0,PHARMA,NaN
77,ICICIPRULI,591.15,791.05,-11.90,15.29,H-MC,22.54,-0.60,33.82,33.82,21.69,X40,ATH,1,0,INSURANCE,0.0
114,NAM-INDIA,632.90,805.90,-2.93,-6.13,M-MC,8.09,-3.26,27.33,27.33,49.71,X40,ATH,1,0,FINANCE,NaN
128,PGHH,14429.90,18249.76,-10.11,-10.34,X-MC,73.99,-0.34,26.47,26.47,0.00,X40,ATH,0,0,FMCG,-1.0
76,ICICIGI,1814.65,2268.71,-3.83,-10.90,H-MC,69.94,-1.06,25.02,25.02,23.23,X40,ATH,1,1,INSURANCE,-1.0
69,HDFCAMC,3959.35,4827.90,-4.22,-6.12,H-MC,47.98,-1.06,21.94,21.94,17.53,X40,ATH,1,0,FINANCE,0.0


In [9]:
# x40n
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40N'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Category,InFolio
145,SUNTV,643.40,1092.00,-12.82,8.58,M-SC,41.04,-1.43,69.72,40.69,13.95,X40N,BTT,1,1,ENTERTAINMENT,0.0
57,FINCABLES,1004.80,1657.02,-22.77,39.13,M-SC,3.47,-1.01,64.91,64.91,20.48,X40N,ATH,1,1,CABLES,NaN
53,EQUITASBNK,68.99,113.10,-14.83,NaN,NaN,90.75,-1.75,63.94,63.94,10.81,X40N,ATH,1,0,BANKS,NaN
147,SYMPHONY,1275.10,2040.93,-3.55,NaN,NaN,64.74,-0.99,60.06,60.06,56.32,X40N,ATH,1,1,DURABLES,0.0
8,ANGELONE,2365.00,3775.05,-10.21,-20.43,H-SC,4.62,-2.73,59.62,59.62,15.78,X40N,ATH,1,1,FINANCE,-1.0
112,MOTILALOFS,708.80,1000.00,-3.83,7.02,H-MC,1.73,-2.41,41.08,116.90,102.38,X40N,NTT,1,1,FINANCE,NaN
165,VINATIORGA,1707.35,2325.26,-7.81,-8.14,X-SC,83.24,-0.73,36.19,36.19,16.61,X40N,ATH,1,0,CHEMICALS,NaN
0,3MINDIA,29429.90,40020.60,-12.35,-27.52,H-MC,54.91,-1.85,35.99,35.99,4.22,X40N,ATH,1,1,MISC,-1.0
24,BAYERCROP,5022.15,6805.00,-15.91,8.15,H-SC,16.18,-1.86,35.50,36.13,2.87,X40N,NTT,1,0,CHEMICALS,0.0
153,TEAMLEASE,2654.85,3487.00,-11.62,-17.20,M-SC,31.21,-1.04,31.34,98.91,1.64,X40N,NTT,1,0,MISC,NaN


In [10]:
# xr, sr, ar
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XR','SR', 'AR'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Category,InFolio
20,BAJAJHIND,28.63,442.93,-20.47,NaN,NaN,36.42,-1.34,1447.08,1447.08,9.65,XR,ATH,1,0,SUGAR,0.0
6,ALOKINDS,19.12,79.81,-23.52,NaN,NaN,15.61,-1.24,317.42,317.42,3.52,AR,ATH,1,0,TEXTILES,NaN
37,CENTRALBK,51.48,190.06,-14.20,-17.40,L-MC,53.76,-1.11,269.19,269.19,9.84,XR,ATH,1,0,BANKS,NaN
9,APEX,243.80,849.16,0.74,NaN,NaN,83.82,-1.67,248.30,248.30,28.21,AR,ATH,1,0,MISC,0.0
130,PNB,98.00,232.89,-14.04,-52.94,L-LC,68.21,-1.44,137.64,137.64,3.21,XR,ATH,1,1,BANKS,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,ULTRACEMCO,11386.20,12083.90,3.32,49.63,X-LC,80.35,-0.30,6.13,6.13,22.32,AR,ATH,1,1,CEMENT,NaN
87,ITBEES,46.73,49.49,11.26,NaN,NaN,78.61,-0.04,5.91,5.91,34.67,AR,ATH,0,0,IT,NaN
113,MUTHOOTFIN,2166.65,2256.90,15.12,39.82,M-MC,91.91,-1.30,4.17,4.17,72.45,XR,ATH,1,1,FINANCE,NaN
162,V2RETAIL,1841.05,1891.00,76.68,NaN,NaN,98.84,5.00,2.71,2.71,440.37,AR,ATH,1,1,TEXTILES,NaN
